In [1]:
import numpy as np

# Constants
S = 6        # Initial stock price
u = 10 / S   # Upward movement factor
d = 5 / S    # Downward movement factor
K = 6        # Strike price
Rf = 1.05    # One plus risk-free rate
T = 1       # Number of time periods

def calculate_stock_prices(S, u, d, T):
    """Calculate stock price tree."""
    # Initialize price tree array
    prices = np.zeros((T+1, T+1))
    prices[0,0] = S
    
    # Fill in stock price tree
    for t in range(1, T+1):
        for j in range(t+1):
            up_moves = j
            down_moves = t - j
            prices[j,t] = S * (u**up_moves) * (d**down_moves)
            
    return prices

def calculate_state_prices(S, u, d, Rf, T):
    """Calculate state prices for multi-period model."""
    # First calculate single period state prices
    X = np.array([[u * S, 1], [d * S, 1]])
    P = np.array([S, 1 / Rf])
    X_inv = np.linalg.inv(X)
    pu, pd = P @ X_inv
    
    # Initialize state price tree
    state_prices = np.zeros((T+1, T+1))
    state_prices[0,0] = 1
    
    # Fill in state price tree using multiplicative property
    for t in range(1, T+1):
        for j in range(t+1):
            up_moves = j
            down_moves = t - j
            state_prices[j,t] = (pu**up_moves) * (pd**down_moves)
            
    return state_prices

def price_multiperiod_option(option_type, S, K, u, d, Rf, T):
    """
    Price a multi-period option.
    
    Parameters:
    option_type: str, either 'call' or 'put'
    """
    # Calculate stock price tree and state prices
    stock_prices = calculate_stock_prices(S, u, d, T)
    state_prices = calculate_state_prices(S, u, d, Rf, T)
    
    # Calculate terminal payoffs
    payoffs = np.zeros(T+1)
    for j in range(T+1):
        if option_type.lower() == 'call':
            payoffs[j] = max(stock_prices[j,T] - K, 0)
        else:  # put option
            payoffs[j] = max(K - stock_prices[j,T], 0)
    
    # Calculate option price using state prices
    option_price = 0
    for j in range(T+1):
        option_price += state_prices[j,T] * payoffs[j]
        
    return option_price


# Calculate and display stock price tree
stock_prices = calculate_stock_prices(S, u, d, T)
print("\nStock Price Tree:")
print(stock_prices)

# Calculate and display state price tree
state_prices = calculate_state_prices(S, u, d, Rf, T)
print("\nState Price Tree:")
print(state_prices)

# Price multi-period options
call_price = price_multiperiod_option('call', S, K, u, d, Rf, T)
put_price = price_multiperiod_option('put', S, K, u, d, Rf, T)

print(f"\nMulti-period Option Prices (T={T}):")
print(f"Call: {call_price:.4f}")
print(f"Put:  {put_price:.4f}")




Stock Price Tree:
[[ 6.  5.]
 [ 0. 10.]]

State Price Tree:
[[1.         0.7047619 ]
 [0.         0.24761905]]

Multi-period Option Prices (T=1):
Call: 0.9905
Put:  0.7048


In [2]:
stock_prices

array([[ 6.,  5.],
       [ 0., 10.]])